<a href="https://colab.research.google.com/github/sammainahkinya1404/East-Africa-Young-Researchers-School-of-Advanced-Machine-Learning-2025/blob/main/Another_copy_of_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install required libraries
!pip install -q transformers sentence-transformers faiss-cpu openai
!pip install -q pdfplumber python-docx pandas

In [ ]:
# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# STEP 3: Import libraries
import faiss
import numpy as np
import pdfplumber
import pandas as pd
from docx import Document
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os


In [ ]:
# STEP 4: Set folder path in Google Drive
folder_path = "/content/drive/MyDrive/Easy_ML/RAN_Group1"

In [ ]:
# STEP 5: Define file extractor
def extract_text(file_path, ext):
    try:
        if ext == "pdf":
            text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
            return text
        elif ext == "docx":
            doc = Document(file_path)
            return "\n".join([p.text for p in doc.paragraphs])
        elif ext == "csv":
            df = pd.read_csv(file_path)
            return df.to_string(index=False)
        else:
            return ""
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

In [ ]:
# STEP 6: Read all supported files in folder
supported_exts = ["pdf", "docx", "csv"]
raw_text = ""

for file in os.listdir(folder_path):
    full_path = os.path.join(folder_path, file)
    ext = file.split(".")[-1].lower()
    if ext in supported_exts:
        print(f"Reading: {file}")
        text = extract_text(full_path, ext)
        raw_text += text + "\n"

# Now raw_text contains the combined text from all files
print(f"\n Total characters extracted: {len(raw_text)}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Easy_ML/RAN_Group1'

In [ ]:
# STEP 7: Split into chunks
def chunk_text(text, max_chars=500):
    paragraphs = text.split("\n")
    chunks, current_chunk = [], ""
    for para in paragraphs:
        if len(current_chunk) + len(para) <= max_chars:
            current_chunk += para + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return [chunk for chunk in chunks if len(chunk) > 50]

doc_chunks = chunk_text(raw_text)

In [ ]:
# STEP 7: Embed chunks using SentenceTransformers (intializing the transformer)
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(doc_chunks, convert_to_tensor=False)
doc_embeddings = np.vstack(doc_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ValueError: need at least one array to concatenate

In [ ]:
# STEP 8: Build FAISS (Facebook AI Similarity Search) index
index = faiss.IndexFlatL2(doc_embeddings.shape[1]) # check the similarity btn the input features and the vector data
index.add(doc_embeddings)

IndexError: tuple index out of range

In [ ]:
# STEP 9: Set your OpenAI API key


In [ ]:
# Updated: RAG using local T5 model (no OpenAI)
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
generator = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

def rag_local_t5_answer(question, top_k=3):
    # Embed query and retrieve
    query_embedding = embedder.encode([question])
    D, I = index.search(np.array(query_embedding), top_k)

    # Combine top passages
    context = "\n".join([doc_chunks[i] for i in I[0]])
    # Increased the number of required paragraphs to three and asked for more detailed paragraphs.
    input_prompt = f"As a Phd student working on this problem, use scientific writing and include in-text citations (APA style) to answer the questions below.\n\nContext:\n{context}\n\nQuestion: {question}"

    # Tokenize and generate
    # Increased max_length for potentially longer responses
    input_ids = tokenizer.encode(input_prompt, return_tensors="pt", truncation=True, max_length=512)
    output_ids = generator.generate(input_ids, max_length=1000000, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# STEP 11: Ask a Question using local T5 model
question = "What are the global challenges we are facing in detection and control of Influenza"
answer = rag_local_t5_answer(question)

print("Question:", question)
print("\n Answer:\n", answer)

Question: What are the global challenges we are facing in detection and control of Influenza

 Answer:
 Epidemiological and virological surveillance Pandemic preparedness


In [ ]:
question = "State three Regions that are highly affected by Influenza"
answer = rag_local_t5_answer(question)

print("Question:", question)
print("\n Answer:\n", answer)

Question: State three Regions that are highly affected by Influenza

 Answer:
 South-East Asia and African Region


In [ ]:
question = "What can we do control and prevent Influenza in these highly affected regions"
answer = rag_local_t5_answer(question)

print("Question:", question)
print("\n Answer:\n", answer)

Question: What can we do control and prevent Influenza in these highly affected regions

 Answer:
 A. Expand seasonal influenza prevention and control Strategic objectives and actions policies and programmes to protect the vulnerable


In [ ]:
question = "Which actions can be adopted to protect the vulnerable from Influenza"
answer = rag_local_t5_answer(question)

print("Question:", question)
print("\n Answer:\n", answer)

Question: Which actions can be adopted to protect the vulnerable from Influenza

 Answer:
 Promote research and innovation for improved and novel diagnostics, vaccines and treatments against influenza


In [ ]:
question = "What is the main goal of WHO in global influence in the years between 2019 and 2022"
answer = rag_local_t5_answer(question)

print("Question:", question)
print("\n Answer:\n", answer)

Question: What is the main goal of WHO in global influence in the years between 2019 and 2022

 Answer:
 Reduce & innovation seasonal influenza Surveillance, burden monitoring & data utilzation Seasonal prevention & control policies & programmes Pandemic preparedness & response GLOBAL INFLUENZA STRATEGY 2019–2030 7 SEVITCEJBO CIGETARTS HIGH-LEVEL OUTCOMES GOALS Minimize zoonotic influenza risk Mitigate pandemic influenza impact
